In [3]:
import os
directory = 'C:\\Users\\giorg\\OneDrive\\Υπολογιστής\\DL Project'
os.chdir(directory)

from Scripts import Utilities as ut
from Scripts import Feature_Extraction as fe
from Scripts import Model_Training as mt
from Scripts import data_loading as dl
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [4]:
dataset = "urbansound8k"
classes = ["children_playing", "drilling", "street_music", "siren", "gun_shot", "car_horn", "air_conditioner", "engine_idling",  "dog_bark", "jackhammer"]

# Pipeline

Download the dataset from the google drive link we provide and put into the folder UrbanSound and everything will run

In [34]:
# Seperate the test folder for it to not be messed in th training process
ut.split_8k_train_test("UrbanSound\\UrbanSound8K\\metadata\\UrbanSound8K.csv", "UrbanSound\\CSVs\\Train_csv", "UrbanSound\\CSVs\\Test_csv")

In [ ]:
# Put files in Train and Test folders
ut.move_train_test_folder("UrbanSound\\CSVs\\Train_csv","UrbanSound\\Train","UrbanSound\\UrbanSound8K\\audio")

ut.move_train_test_folder("UrbanSound\\CSVs\\Test_csv","UrbanSound\\Test","UrbanSound\\UrbanSound8K\\audio")

In [ ]:
# In train folder put evry file in the corespondiong category
ut.move_to_class_folder("UrbanSound\\CSVs\\Train_csv","UrbanSound\\Train",dataset)

# Neural_Network


In [ ]:
fe.NN_Features("UrbanSound\\Train", "UrbanSound\\Features\\NN_Features")


In [10]:
x_train, x_val, y_train, y_val =  dl.load_data("UrbanSound\\Features\\NN_Features", dataset)

In [29]:

model = mt.create_model_NN()
learning_rate = 0.0001
model.compile(optimizer=Adam(learning_rate = learning_rate ), loss='categorical_crossentropy', metrics=[mt.f1_score, 'accuracy'])

early_stopping = EarlyStopping(monitor='val_f1_score', mode='max', patience=15, verbose=1)

In [ ]:
hist = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=400, callbacks = early_stopping
    )


In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
mt.plot_training_history(acc,val_acc,"accuracy")

In [ ]:
mt.plot_confusion_matrix(model,x_val,y_val,classes)

In [ ]:
model.save("Models\\Neural_Network_Audioset_Simple_NN.h5")

# CNN

In [6]:
fe.CNN_Features_mel("UrbanSound\Train", classes, "UrbanSound\Features\CNN_Features")


No files failed.


In [25]:
x_train, x_val, y_train, y_val =  dl.load_data("UrbanSound\Features\CNN_Features", dataset)

In [ ]:
model = mt.create_model_CNN(num_classes=10)
learning_rate = 0.0001
model.compile(optimizer=Adam(learning_rate = learning_rate ), loss='categorical_crossentropy', metrics=[mt.f1_score, 'accuracy'])

early_stopping = EarlyStopping(monitor='val_f1_score', mode='max', patience=15, verbose=1)

In [ ]:
hist = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50, callbacks = early_stopping,
    batch_size=32
    )

In [ ]:
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
mt.plot_training_history(acc,val_acc,"accuracy")

In [ ]:
mt.plot_confusion_matrix(model,x_val,y_val,classes)

In [ ]:
model.save("Models\\CNN_Simple_UrbanSound.h5")